In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#get info from csv in weatherpy
cities_pd = pd.read_csv("../weatherpy/output_data/city_weather.csv")
cities_pd.head()


,Unnamed: 0,City,Temperature,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date
0,0,tura,72.64,90,90.22,25.52,0,IN,1.63,1603055527
1,1,barrow,21.20,85,-156.79,71.29,75,US,11.41,1603055561
2,2,kodiak,44.60,75,-152.41,57.79,75,US,10.29,1603055561
3,3,muros,57.99,84,-9.06,42.78,99,ES,7.45,1603055562
4,4,ushuaia,41.00,80,-68.30,-54.80,75,AR,29.97,1603055358


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Plot Heatmap
fig = gmaps.figure()

# Store latitude, longitude and Humidity in locations
locations = cities_pd[["Latitude", "Longitude"]]
Humidity = cities_pd["Humidity"]

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#create new data with specific conditions
ideal_cities = cities_pd.loc[(cities_pd["Temperature"]>60)&(cities_pd["Temperature"]<90)&(cities_pd["Windspeed"]<20)&(cities_pd["Cloudiness"]<50)].dropna()
ideal_cities

,Unnamed: 0,City,Temperature,Humidity,Longitude,Latitude,Cloudiness,Country,Windspeed,Date
0,0,tura,72.64,90,90.22,25.52,0,IN,1.63,1603055527
5,5,huarmey,68.72,66,-78.15,-10.07,41,PE,11.18,1603055563
10,10,georgetown,80.60,83,100.34,5.41,20,MY,3.36,1603055334
13,13,saint george,82.99,10,-113.58,37.10,1,US,9.44,1603055341
14,14,pinotepa de don luis,84.04,63,-97.92,16.42,15,MX,5.48,1603055567
...,...,...,...,...,...,...,...,...,...,...
518,518,hualmay,66.36,70,-77.61,-11.10,24,PE,9.44,1603055764
519,519,abu dhabi,80.60,51,54.37,24.47,0,AE,5.82,1603055764
520,520,vallenar,81.68,23,-70.76,-28.57,0,CL,6.96,1603055765
529,529,charters towers,67.77,77,146.27,-20.10,0,AU,9.84,1603055768


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
#create new df *copy to avoid the error message
hotel_df = ideal_cities[["City","Country","Latitude", "Longitude"]].copy()
#add hotel name column
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,tura,IN,25.52,90.22,
5,huarmey,PE,-10.07,-78.15,
10,georgetown,MY,5.41,100.34,
13,saint george,US,37.10,-113.58,
14,pinotepa de don luis,MX,16.42,-97.92,
...,...,...,...,...,...
518,hualmay,PE,-11.10,-77.61,
519,abu dhabi,AE,24.47,54.37,
520,vallenar,CL,-28.57,-70.76,
529,charters towers,AU,-20.10,146.27,


In [14]:
# find the closest hotels with params using gkey
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {  
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
}
# use iterrows to iterate through dataframe
for index, row in hotel_df.iterrows():

    # get location from df
    latitude = row['Latitude']
    longitude = row['Longitude']

    # add keyword to params dict
    params['location'] = f'{latitude},{longitude}'

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {row['City']} hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 0: tura.
Closest tura hotel is Hotel RIKMAN Continental.
------------
Retrieving Results for Index 5: huarmey.
Closest huarmey hotel is Hotel Miramar.
------------
Retrieving Results for Index 10: georgetown.
Closest georgetown hotel is Cititel Penang.
------------
Retrieving Results for Index 13: saint george.
Closest saint george hotel is Best Western Plus Abbey Inn.
------------
Retrieving Results for Index 14: pinotepa de don luis.
Missing field/result... skipping.
------------
Retrieving Results for Index 18: faanui.
Closest faanui hotel is Bora-Bora Pearl Beach Resort & Spa.
------------
Retrieving Results for Index 23: souillac.
Closest souillac hotel is Bils' Villa.
------------
Retrieving Results for Index 40: amealco.
Closest amealco hotel is Hotel Jose Maria.
------------
Retrieving Results for Index 42: bambous virieux.
Closest bambous virieux hotel is Casa Tia Villa.
------------
Retrieving Results for Index 45: dalmau.
Closest dalmau hotel is 

Closest marietta hotel is Hilton Atlanta/Marietta Hotel & Conference Center.
------------
Retrieving Results for Index 314: nouadhibou.
Closest nouadhibou hotel is El Medina.
------------
Retrieving Results for Index 321: fortuna.
Closest fortuna hotel is Super 8 by Wyndham Fortuna.
------------
Retrieving Results for Index 324: do gonbadan.
Closest do gonbadan hotel is مهمانسرا و تالار فرهنگیان شهرستان گچساران.
------------
Retrieving Results for Index 325: durango.
Missing field/result... skipping.
------------
Retrieving Results for Index 326: fomboni.
Closest fomboni hotel is Auberge les abou.
------------
Retrieving Results for Index 330: saint andrews.
Closest saint andrews hotel is Baymont by Wyndham Columbia Northwest.
------------
Retrieving Results for Index 333: half moon bay.
Closest half moon bay hotel is The Ritz-Carlton, Half Moon Bay.
------------
Retrieving Results for Index 335: maamba.
Missing field/result... skipping.
------------
Retrieving Results for Index 336: d

In [15]:
#df with set columns and name of hotels
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
0,tura,IN,25.52,90.22,Hotel RIKMAN Continental
5,huarmey,PE,-10.07,-78.15,Hotel Miramar
10,georgetown,MY,5.41,100.34,Cititel Penang
13,saint george,US,37.10,-113.58,Best Western Plus Abbey Inn
14,pinotepa de don luis,MX,16.42,-97.92,
...,...,...,...,...,...
518,hualmay,PE,-11.10,-77.61,Hotel Restaurant Turístico Casablanca
519,abu dhabi,AE,24.47,54.37,Beach Rotana
520,vallenar,CL,-28.57,-70.76,Humacao Bed & Breakfast
529,charters towers,AU,-20.10,146.27,Cattleman's Rest Motor Inn


In [17]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [18]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add layer
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))